In [2]:
import mysql.connector

mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql')

cur = mydb.cursor()

In [3]:
query = "create database if not exists ecommerce"
cur.execute(query)

In [4]:
mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql',
                               database = "ecommerce" )

cur = mydb.cursor()

query = """
create table if not exists customers(
customer_id int primary key,
name varchar(30)
)
"""

cur.execute(query)
mydb.commit()
cur.close()

True

In [5]:
mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql',
                               database = "ecommerce" )

cur = mydb.cursor()

query = """
create table if not exists orders(
order_id int primary key,
customer_id int,
order_date DATE,
amount decimal(10,2),
foreign key (customer_id) references customers(customer_id)
)
"""

cur.execute(query)
mydb.commit()
cur.close()

True

In [6]:
mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql',
                               database = "ecommerce" )

cur = mydb.cursor()

query = """
insert into customers (customer_id,name) values
(1,"Rahul") , (2,"Amit") , (3,"Abhay") , (3,"Ramesh") on duplicate key update name = values(name)
"""
cur.execute(query)
mydb.commit()
cur.close()

True

In [7]:
mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql',
                               database = "ecommerce" )

cur = mydb.cursor()

query = """
select * from customers
"""
cur.execute(query)
result = cur.fetchall()
for customer in result:
    print(customer)

mydb.commit()
cur.close()

(1, 'Rahul')
(2, 'Amit')
(3, 'Ramesh')


True

In [10]:
mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql',
                               database = "ecommerce" )

cur = mydb.cursor()

query = """
insert into orders(order_id,customer_id,order_date,amount) values
(101,1,'2024-01-01',120.0),
(102,1,'2024-02-05',180.0),
(103,2,'2024-01-10',75.0),
(104,3,'2024-03-01',150.0),
(105,2,'2024-03-15',200.0),
(106,1,'2024-04-01',100.0)
"""

cur.execute(query)

mydb.commit()
cur.close()

True

In [11]:
mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql',
                               database = "ecommerce" )

cur = mydb.cursor()

query = """
select * from orders
"""
cur.execute(query)
result = cur.fetchall()
for order in result:
    print(order)

mydb.commit()
cur.close()

(101, 1, datetime.date(2024, 1, 1), Decimal('120.00'))
(102, 1, datetime.date(2024, 2, 5), Decimal('180.00'))
(103, 2, datetime.date(2024, 1, 10), Decimal('75.00'))
(104, 3, datetime.date(2024, 3, 1), Decimal('150.00'))
(105, 2, datetime.date(2024, 3, 15), Decimal('200.00'))
(106, 1, datetime.date(2024, 4, 1), Decimal('100.00'))


True

In [65]:
query = """
select customer_id , min(order_date),max(order_date) from orders group by customer_id
"""
cur.execute(query)
result = cur.fetchall()

for res in result:
    print(res)

(1, datetime.date(2024, 1, 1), datetime.date(2024, 4, 1))
(2, datetime.date(2024, 1, 10), datetime.date(2024, 3, 15))
(3, datetime.date(2024, 3, 1), datetime.date(2024, 3, 1))


**Calculate RFM Metrics**

In [36]:
mydb = mysql.connector.connect(host = 'localhost',
                               user = 'root',
                               password = '725@Mysql',
                               database = "ecommerce" )

cur = mydb.cursor()

query = """
create temporary table temp_recency as
select customer_id, DATEDIFF(curdate() , max(order_date)) as recency
from orders
group by customer_id
"""

cur.execute(query)

In [37]:
query = "select * from temp_recency"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)

(1, 84)
(2, 101)
(3, 115)


In [38]:
query = """
create temporary table temp_frequency as
select customer_id, count(order_id) as frequency
from orders
group by customer_id
"""

cur.execute(query)

In [39]:
query = "select * from temp_frequency"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)

(1, 3)
(2, 2)
(3, 1)


In [40]:
query = """
create temporary table temp_monetary as
select customer_id, sum(amount) as monetary
from orders
group by customer_id
"""

cur.execute(query)

In [41]:
query = "select * from temp_monetary"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)

(1, Decimal('400.00'))
(2, Decimal('275.00'))
(3, Decimal('150.00'))


In [42]:
query = """

create temporary table temp_rfms as 
select r.customer_id, r.recency, f.frequency, m.monetary
from temp_recency r
join temp_frequency f on r.customer_id = f.customer_id
join temp_monetary m on r.customer_id = m.customer_id

"""
cur.execute(query)


In [43]:
query = "select * from temp_rfms"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)



(1, 84, 3, Decimal('400.00'))
(2, 101, 2, Decimal('275.00'))
(3, 115, 1, Decimal('150.00'))


**Segment customers based on RFMS scores**

In [45]:
query = """
create temporary table temp_recency_score as
select customer_id, recency,
case
    when recency <= 30 then 5
    when recency <= 60 then 4
    when recency <= 90 then 3
    when recency <= 120 then 2
    else 1
    end as recency_score
from temp_recency
"""
cur.execute(query)

In [46]:
query = "select * from temp_recency_score"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)

(1, 84, 3)
(2, 101, 2)
(3, 115, 2)


In [47]:
#Assign frequency score

query = """
create temporary table temp_frequency_score as
select customer_id, frequency,
case
    when frequency >= 10 then 5
    when frequency >= 7 then 4
    when frequency >= 4 then 3
    when frequency >= 2 then 2
    else 1
    end as frequency_score
from temp_frequency
"""
cur.execute(query)

In [48]:
query = "select * from temp_frequency_score"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)

(1, 3, 2)
(2, 2, 2)
(3, 1, 1)


In [49]:
#Assign Monetary score


query = """
create temporary table temp_monetary_score as
select customer_id, monetary,
case
    when monetary >= 1000 then 5
    when monetary >= 750 then 4
    when monetary >= 500 then 3
    when monetary >= 250 then 2
    else 1
    end as monetary_score
from temp_monetary
"""
cur.execute(query)

In [50]:
query = "select * from temp_monetary_score"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)

(1, Decimal('400.00'), 2)
(2, Decimal('275.00'), 2)
(3, Decimal('150.00'), 1)


In [51]:
#Combine RFM scores

query = """
create temporary table temp_rfm_scores as
select r.customer_id, r.recency , r.recency_score,
f.frequency,f.frequency_score,
m.monetary,m.monetary_score,
(r.recency_score + f.frequency_score + m.monetary_score)
as total_rfm_score 
from temp_recency_score r
join temp_frequency_score f on r.customer_id = f.customer_id
join temp_monetary_score m on r.customer_id = m.customer_id

"""
cur.execute(query)

In [57]:
query = "select * from temp_rfm_scores"

cur.execute(query)

result = cur.fetchall()
for res in result:
    print(res)

(1, 84, 3, 3, 2, Decimal('400.00'), 2, 7)
(2, 101, 2, 2, 2, Decimal('275.00'), 2, 6)
(3, 115, 2, 1, 1, Decimal('150.00'), 1, 4)


**Derive Insights**

In [61]:
#High Potential customer
query = """
select * from temp_rfm_scores
where total_rfm_score > 6
"""

cur.execute(query)

best_customer = cur.fetchall()
for customer in best_customer:
    print(res)

(3, 115, 2, 1, 1, Decimal('150.00'), 1, 4)


In [62]:
#Customer at risk

query = """
select * from temp_rfm_scores
where total_rfm_score < 5
"""

cur.execute(query)

customer_at_risk = cur.fetchall()
for customer in customer_at_risk:
    print(res)

(3, 115, 2, 1, 1, Decimal('150.00'), 1, 4)
